In [9]:
import pandas as pd
import numpy as np
import pickle
from joblib import Parallel, delayed

In [29]:
with open('/mnt/spare8tb/all_dataframes_sequential.pkl','rb') as f:
    all_data = pickle.load(f)
with open('updated_dividend_data.pkl', 'rb') as file:
    divident = pickle.load(file)
with open('updated_eps_data.pkl', 'rb') as file:
    eps = pickle.load(file)
with open('updated_shares_outstanding_data_250.pkl', 'rb') as file:
    shares = pickle.load(file)

In [7]:
def paralelizer(data, year):
    ticker = data.columns[0].split('_')[0]
    if ticker in ["TLT","IEF", "LQD", "AGG", "DBC", "GSG", "GLD", "BND", "VNQ", "HYG", "EFA",'SLV','UCO','DBA']:
        return {f'{ticker}':0}
    data=data[(data.index.year<=year)&(data.index.year>=year-1)]
    data.sort_index(inplace=True)
    if len(data)<=245*2:
        return {f'{ticker}':0}
    try:
        data=data[(data.index.year==year)]
        data.reset_index(inplace=True)
        DV=data[f'{ticker}_vw']*data[f'{ticker}_Volume']
        medianDailyVolume=DV.quantile(0.75)
        return {f'{ticker}':medianDailyVolume}
    except:
        return {f'{ticker}':0}
years_since_2005 = list(range(2004, 2024))
years_since_2005[1:-1]

[2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020,
 2021,
 2022]

In [10]:
stockdict1= {}
for year in years_since_2005:
    print(year)
    # stocklistInner=dict()
    fitnesses = Parallel(n_jobs=28)(delayed(paralelizer)(i, year) for i in all_data)
    keys = []
    values = []
    
    for item in fitnesses:
        key, value = item.popitem()
        keys.append(key)
        values.append(value)
    
    resDf = pd.DataFrame({'Key': keys, 'Value': values})
    resDf = resDf[resDf['Value']>0]
    
    sorted_stocklist = resDf.sort_values('Value', ascending=False).iloc[:25, :]
    top_50_tickers = list(sorted_stocklist['Key'])[:50]
    
    stockdict1[year]=sorted_stocklist
    pickle.dump(stockdict1, open('stockdict1.pkl', 'wb'))



2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023


In [11]:
for i in list(stockdict1.keys()):
    if i >= 2004:
        print('-----------')
        print(i)
        for idx, row in stockdict1[i].iterrows():
            print(row['Key'])

-----------
2004
-----------
2005
-----------
2006
SPY
QQQQ
GOOG
IWM
AAPL
MSFT
OIH
INTC
XLE
XOM
CSCO
DIA
GE
GS
PFE
C
YHOO
SMH
COP
QCOM
SLB
WMT
ORCL
MO
SNDK
-----------
2007
SPY
IWM
QQQQ
AAPL
GOOG
GS
EEM
XOM
XLF
DIA
MSFT
RIMM
C
XLE
OIH
INTC
CSCO
GE
BIDU
BAC
FCX
PFE
JPM
MER
COP
-----------
2008
SPY
QQQQ
IWM
AAPL
XLF
EEM
GOOG
XOM
SDS
DIA
XLE
MSFT
C
QID
GS
RIMM
BAC
GE
JPM
POT
WFC
BIDU
OIH
CSCO
INTC
-----------
2009
SPY
QQQQ
BAC
IWM
SDS
AAPL
SKF
EEM
C
JPM
GS
WFC
XOM
XLF
DIA
GE
GOOG
SSO
MSFT
XLE
IYR
RIMM
INTC
EWZ
CSCO
-----------
2010
SPY
AAPL
IWM
QQQQ
EEM
BAC
C
FAS
XOM
GOOG
GS
MSFT
JPM
XLF
INTC
SDS
F
EWZ
CSCO
GE
WFC
FCX
XLE
FXI
LVS
-----------
2011
SPY
AAPL
IWM
EEM
BAC
C
GOOG
XOM
MSFT
XLE
JPM
INTC
XLF
AMZN
CSCO
NFLX
BIDU
GE
EWZ
DIA
VWO
TNA
VXX
WFC
FCX
-----------
2012
SPY
AAPL
IWM
EEM
BAC
GOOG
MSFT
C
JPM
XOM
XLF
INTC
XLE
VXX
GE
AMZN
WFC
VWO
EWZ
T
ORCL
DIA
CSCO
IBM
QCOM
-----------
2013
SPY
AAPL
IWM
EEM
QQQ
GOOG
BAC
MSFT
TSLA
C
XOM
JPM
DIA
VXX
GDX
NFLX
XLF
GE
PFE
CSCO
INTC
XLE
AMZN
T
IYR
---

In [13]:
yearStocks={key: [] for key in [2005+1, 2006+1, 2007+1, 2008+1,2009+1, 2010+1,2011+1, 2012+1, 2013+1, 2014+1,
2015+1,
2016+1,
2017+1,
2018+1,
2019+1,
2020+1,
2021+1,
2022+1,2023+1]}

for i in list(stockdict1.keys()):
    if i >= 2005:
        print('-----------')
        print(i)
        for idx, row in stockdict1[i].iterrows():
            yearStocks[i+1].append(row['Key'])

-----------
2005
-----------
2006
-----------
2007
-----------
2008
-----------
2009
-----------
2010
-----------
2011
-----------
2012
-----------
2013
-----------
2014
-----------
2015
-----------
2016
-----------
2017
-----------
2018
-----------
2019
-----------
2020
-----------
2021
-----------
2022
-----------
2023


In [14]:
pickle.dump(yearStocks, open('stockstobeused1.pkl', 'wb'))

In [15]:
with open('stockstobeused1.pkl','rb') as f:
    year = pickle.load(f)
year

{2006: [],
 2007: ['SPY',
  'QQQQ',
  'GOOG',
  'IWM',
  'AAPL',
  'MSFT',
  'OIH',
  'INTC',
  'XLE',
  'XOM',
  'CSCO',
  'DIA',
  'GE',
  'GS',
  'PFE',
  'C',
  'YHOO',
  'SMH',
  'COP',
  'QCOM',
  'SLB',
  'WMT',
  'ORCL',
  'MO',
  'SNDK'],
 2008: ['SPY',
  'IWM',
  'QQQQ',
  'AAPL',
  'GOOG',
  'GS',
  'EEM',
  'XOM',
  'XLF',
  'DIA',
  'MSFT',
  'RIMM',
  'C',
  'XLE',
  'OIH',
  'INTC',
  'CSCO',
  'GE',
  'BIDU',
  'BAC',
  'FCX',
  'PFE',
  'JPM',
  'MER',
  'COP'],
 2009: ['SPY',
  'QQQQ',
  'IWM',
  'AAPL',
  'XLF',
  'EEM',
  'GOOG',
  'XOM',
  'SDS',
  'DIA',
  'XLE',
  'MSFT',
  'C',
  'QID',
  'GS',
  'RIMM',
  'BAC',
  'GE',
  'JPM',
  'POT',
  'WFC',
  'BIDU',
  'OIH',
  'CSCO',
  'INTC'],
 2010: ['SPY',
  'QQQQ',
  'BAC',
  'IWM',
  'SDS',
  'AAPL',
  'SKF',
  'EEM',
  'C',
  'JPM',
  'GS',
  'WFC',
  'XOM',
  'XLF',
  'DIA',
  'GE',
  'GOOG',
  'SSO',
  'MSFT',
  'XLE',
  'IYR',
  'RIMM',
  'INTC',
  'EWZ',
  'CSCO'],
 2011: ['SPY',
  'AAPL',
  'IWM',
  'QQQQ',
 

In [16]:
a = year.values()
stock_lists = a
combined_stocks = list(set(stock for sublist in stock_lists for stock in sublist))
combined_stocks = combined_stocks
combined_stocks.sort()  # Optional, if you want the list to be sorted
print(combined_stocks)
print(len(combined_stocks))

['AAPL', 'AMC', 'AMD', 'AMZN', 'AVGO', 'BA', 'BABA', 'BAC', 'BIDU', 'BRK.A', 'C', 'COP', 'CSCO', 'DIA', 'DIS', 'EEM', 'EWZ', 'F', 'FAS', 'FB', 'FCX', 'FXI', 'GDX', 'GE', 'GILD', 'GOOG', 'GOOGL', 'GS', 'IBM', 'INTC', 'IVV', 'IWM', 'IYR', 'JPM', 'LVS', 'MER', 'MO', 'MRNA', 'MSFT', 'MU', 'NFLX', 'NIO', 'NVDA', 'OIH', 'ORCL', 'PCLN', 'PFE', 'POT', 'PYPL', 'QCOM', 'QID', 'QQQ', 'QQQQ', 'RIMM', 'ROKU', 'SDS', 'SHOP', 'SKF', 'SLB', 'SMH', 'SNDK', 'SPY', 'SQ', 'SQQQ', 'SSO', 'T', 'TNA', 'TQQQ', 'TSLA', 'UNH', 'V', 'VOO', 'VWO', 'VXX', 'VZ', 'WFC', 'WMT', 'XLE', 'XLF', 'XOM', 'YHOO']
81


In [44]:
note = []
ine = []
for name in combined_stocks:
    if name in eps:
        print('in the list',name)
        ine.append(name)
    else:
        print('not in the list',name)
        note.append(name)

in the list AAPL
in the list AMC
in the list AMD
in the list AMZN
not in the list AVGO
in the list BA
in the list BABA
in the list BAC
in the list BIDU
in the list BRK.A
in the list C
in the list COP
in the list CSCO
not in the list DIA
in the list DIS
not in the list EEM
not in the list EWZ
in the list F
not in the list FAS
in the list FB
in the list FCX
not in the list FXI
not in the list GDX
in the list GE
in the list GILD
in the list GOOG
in the list GOOGL
in the list GS
in the list IBM
in the list INTC
not in the list IVV
not in the list IWM
not in the list IYR
in the list JPM
in the list LVS
not in the list MER
in the list MO
in the list MRNA
in the list MSFT
in the list MU
in the list NFLX
in the list NIO
in the list NVDA
not in the list OIH
in the list ORCL
in the list PCLN
in the list PFE
in the list POT
in the list PYPL
in the list QCOM
not in the list QID
not in the list QQQ
not in the list QQQQ
in the list RIMM
in the list ROKU
not in the list SDS
in the list SHOP
not in th

In [32]:
len(inl)

51

In [33]:
len(notl)

30

In [38]:
len(ine)

52

In [39]:
len(note)

29

In [45]:
len(ind)

51

In [46]:
len(notd)

30

In [47]:
notl

['AVGO',
 'DIA',
 'EEM',
 'EWZ',
 'FAS',
 'FXI',
 'GDX',
 'IVV',
 'IWM',
 'IYR',
 'MER',
 'OIH',
 'POT',
 'QID',
 'QQQ',
 'QQQQ',
 'SDS',
 'SKF',
 'SMH',
 'SNDK',
 'SPY',
 'SQQQ',
 'SSO',
 'TNA',
 'TQQQ',
 'VOO',
 'VWO',
 'VXX',
 'XLE',
 'XLF']

In [48]:
#Except AVGO everything is etf and MER, POT, QQQQ, SNDK de listed

In [55]:
ticker = 'NVDA'

In [56]:
eps[ticker].head(5)

,reportDate,epsActual
0,2025-05-20,NaN
1,2025-02-19,NaN
2,2024-11-20,0.810
3,2024-08-28,0.680
4,2024-05-22,0.612


In [57]:
shares[ticker].tail(5)

,Date,Shares Outstanding
98,2023-12-31,24940000000
99,2024-03-31,24900000000
100,2024-06-30,24890000000
101,2024-09-30,24848000000
102,2024-12-31,24774000000


In [58]:
divident[ticker].tail(5)

Date
2023-12-05    0.004
2024-03-05    0.004
2024-06-11    0.010
2024-09-12    0.010
2024-12-05    0.010
Name: Dividends, dtype: float64

In [59]:
for idx,i in enumerate(all_data):
    ticker = i.columns[0].split('_')[0]
    if ticker == 'NVDA':
        print(ticker,idx)
        
    
    
        

NVDA 17211


In [61]:
window_data = all_data[17211]
window_data.reset_index(inplace=True)
window_data

,Date,NVDA_Volume,NVDA_vw,NVDA_Open,NVDA_Close,NVDA_High,NVDA_Low,NVDA_n
0,2005-01-03,1.068353e+09,0.1997,0.2032,0.1965,0.2060,0.1957,23852
1,2005-01-04,7.846225e+08,0.1896,0.1979,0.1873,0.1983,0.1843,18915
2,2005-01-05,7.251280e+08,0.1888,0.1853,0.1890,0.1913,0.1843,18441
3,2005-01-06,5.695403e+08,0.1889,0.1924,0.1872,0.1946,0.1858,13339
4,2005-01-07,7.630102e+08,0.1853,0.1876,0.1836,0.1890,0.1809,17662
...,...,...,...,...,...,...,...,...
5022,2024-12-16,2.284516e+08,131.7878,134.1800,132.0000,134.4000,130.4200,1985180
5023,2024-12-17,2.514261e+08,129.2877,129.0900,130.3900,131.5900,126.8600,2135313
5024,2024-12-18,2.663488e+08,133.0844,133.8600,128.9100,136.7000,128.2800,2090684
5025,2024-12-19,1.984336e+08,131.5037,131.7600,130.6800,134.0300,129.5500,1597979


In [65]:
shar = shares['NVDA']
window_data['Date'] = pd.to_datetime(window_data['Date'])
shar['Date'] = pd.to_datetime(shar['Date'])
shar = shar.sort_values('Date')
daily_dates = pd.date_range(start=shar['Date'].min(), end=shar['Date'].max(), freq='D')
daily_df = pd.DataFrame(daily_dates, columns=['Date'])
daily_df = pd.merge(daily_df, shar, on='Date', how='left')
daily_df['Shares Outstanding'] = daily_df['Shares Outstanding'].fillna(method='ffill')
combined_df = pd.merge(daily_df,window_data,on='Date')
combined_df


,Date,Shares Outstanding,NVDA_Volume,NVDA_vw,NVDA_Open,NVDA_Close,NVDA_High,NVDA_Low,NVDA_n
0,2005-01-03,2.073391e+10,1.068353e+09,0.1997,0.2032,0.1965,0.2060,0.1957,23852
1,2005-01-04,2.073391e+10,7.846225e+08,0.1896,0.1979,0.1873,0.1983,0.1843,18915
2,2005-01-05,2.073391e+10,7.251280e+08,0.1888,0.1853,0.1890,0.1913,0.1843,18441
3,2005-01-06,2.073391e+10,5.695403e+08,0.1889,0.1924,0.1872,0.1946,0.1858,13339
4,2005-01-07,2.073391e+10,7.630102e+08,0.1853,0.1876,0.1836,0.1890,0.1809,17662
...,...,...,...,...,...,...,...,...,...
5022,2024-12-16,2.484800e+10,2.284516e+08,131.7878,134.1800,132.0000,134.4000,130.4200,1985180
5023,2024-12-17,2.484800e+10,2.514261e+08,129.2877,129.0900,130.3900,131.5900,126.8600,2135313
5024,2024-12-18,2.484800e+10,2.663488e+08,133.0844,133.8600,128.9100,136.7000,128.2800,2090684
5025,2024-12-19,2.484800e+10,1.984336e+08,131.5037,131.7600,130.6800,134.0300,129.5500,1597979
